In [1]:
import os
import numpy as np

## loads text file into python
#base_path = "C:/Users/ChrisC" ## this should be set to where-ever the text files are kept. Make sure to use forward slashes
#base_path = "/Users/ami_zou/Desktop/COMP562_Project"
base_path = ""
filename = "V.STORM VS EG.txt" ## instead of back slashes

path_to_file = os.path.join(base_path, filename)
fd = open(path_to_file, 'r', encoding ="utf8")

## loads the text file into a double array with each message being an index in the first array and each word in the message
## as an index in the second array
lines = fd.read().split('\n')

## counter for how many messages are in the chatlog
m = 0

for k in range(len(lines)):
    lines[k] = lines[k].split(' ')
    m = m + 1
    
## wordl will be a copy of lines without timestamps or usernames. words will be a list of the words that appear in the chatlog
wordl = lines.copy()
words = []
    
## removes the timestamp and the user name, which are stored in the first two elements of each sub-array
for k in range(len(wordl)):
    wordl[k] = wordl[k][2:]

## adds each word to the words array. A word will only be added once, regardless of how many times it appears
for k in range(len(wordl)):
    for w in wordl[k]:
        if words.count(w) == 0:
            words.append(w)

## creates room for parameters, add more zeros as needed
## characters in the word is set in index 2
for k in range(len(words)):
    words[k] = [words[k], 0, len(words[k]), 0, 0, 0, 0]
    

In [2]:
## counts number of time each word occurs in the chat(set in index 1)
def wordOccur(w, wordList):
    c = 0
    for k in range(len(wordList)):
        c = c + wordList[k].count(w)
            
    return c

def wordOccurAvg(w, b, wordList):
    c = 0
    for k in range(len(wordList)):
        if wordList[k].count(w) != 0:
            c = c + 1
    #TODO: UPDATE IT HERE: Can't divide by 0
    if c == 0:
        return 0
    else:
        return b/c
        

In [3]:
## counts number of capital letters per word
def wordCaps(word):
    c = 0
    
    for n in word:
        if n.isupper():
            c = c + 1
    return c

## counts number of lower case letters before captial 
def lowersBeforeUpper(word):
    counter = 0
    
    if not word[0].isalpha():
        word = word[1:]
    
    for letter in word:
        if not letter.isupper():
            counter += 1
        else:
            break
    if counter == len(word):
        return 0
    else:
        return counter
    
def singleWord(word, wordsl):
    c = 0
    for k in range(len(wordsl)):
        if wordsl[k].count(word) > 0 and len(wordsl[k]) == 1:
            c += 1
    return c


In [4]:
## index 1 is set to the number of occurances   
## index 2 is set to the length of the word
## index 3 is set to the number of letters that are capitalized
## index 4 is set to average number of repetitions per message
## index 5 is set to the number of lower case letters before the first capitalized letter, zero if there are no capitalizations
## or if the entire word is capitalized
## index 6 is set to the number of times the word appears in a message by itself.

def putTogether(words, wordl):
    for k in range(len(words)):
        words[k][1] = wordOccur(words[k][0], wordl)
        words[k][3] = wordCaps(words[k][0])
        words[k][4] = wordOccurAvg(words[k][0], words[k][1], wordl)
        words[k][5] = lowersBeforeUpper(words[k][0])
        words[k][6] = singleWord(words[k][0], wordl)
        
    return words

In [5]:
words1 = putTogether(words, wordl)
fd.close()
## Creates a list of all global emotes used by Twitch.tv and the extremely popular plug in BTTV to train the model on.
filename2 = "GlobalEmotes.txt"

path_to_file2 = os.path.join(base_path, filename2)
fd2 = open(path_to_file2, 'r', encoding ="utf8")

emotes = fd2.read().split('\n')
emoteList = []

for k in range(len(emotes)):
    if len(emotes[k]) >= 2:
        emoteList.append(emotes[k])

fd2.close()

filename3 = "BTTVEmotes.txt"

path_to_file3 = os.path.join(base_path, filename3)
fd3 = open(path_to_file3, 'r', encoding ="utf8")

bttvEmotes = fd3.read().split('\n')
        
for k in range(len(bttvEmotes)):
    bttvEmotes[k] = bttvEmotes[k].split(' ')
    if (len(bttvEmotes[k][0]) > 1):
        emoteList.append(bttvEmotes[k][0])
fd3.close()

In [6]:
## appends certain known channel emotes to master list of emotes. This list is far from comprehensive, and will require
## training the model to be able to recognize such emotes. The point is for this model to be rebust and able to detect
## future emotes that are not part of this list, as emotes are regularly added, deleted and replaced
emoteList.append("admiralG")
emoteList.append("admiralW")
emoteList.append("admiralC1")
emoteList.append("admiralC2")
emoteList.append("admiralGame")
emoteList.append("admiralGa")
emoteList.append("admiralC")
emoteList.append("cpkSwag")
emoteList.append("cpkFiesta")
emoteList.append("cpkMurica")
emoteList.append("febbyW")
emoteList.append("febbyS")
emoteList.append("febbyNa")
emoteList.append("febbyWh")
emoteList.append("febbyGasm")
emoteList.append("febbyWut")
emoteList.append("febbyS1")
emoteList.append("febbyP")
emoteList.append("capES1")
emoteList.append("capES2")
emoteList.append("capES3")
emoteList.append("capCap")

In [7]:
from collections import deque
## Converts out list of words into a 4542 x 6 array with numpy minus the initial word in index 1
words2 = words1.copy()
for k in range(len(words2)):            ## iterates through a copy of the word list and removes the word at index 0
    words2[k] = deque(words2[k])
    words2[k].popleft()
    
wordArray = np.asarray(words2)         

## Creates a 6 x 1 Matrix to hold weights
weights = np.random.randn(wordArray.shape[1])*0.001
print(weights)

## Creates a 4542 x 1 Matrix to hold outputs
output = np.ones(wordArray.shape[0])

for k in range(output.shape[0]):
    if emoteList.count(words[k][0]) >= 1:
        output[k] = 1
    else:
        output[k] = 0    

[-1.08036291e-03  7.66315042e-04  9.61938498e-04  6.38333722e-04
  4.05485940e-05 -5.57014930e-05]


In [8]:
## implementing log-likelihood function for {0, 1}
def loglikelihood(w, X,y, alpha):
    xbMatrix = np.dot(X, w)
    prob = np.exp(y * xbMatrix)/(1 + np.exp(xbMatrix))
    X = X.T
    gradVal = np.dot(X, np.exp(xbMatrix)/(1 + np.exp(xbMatrix)))
    gradValy = np.dot(X, y)
    penalty = alpha/2 * np.sum(w**2)
    gradPenalty = -alpha * w
    return -np.sum(y * (xbMatrix) - np.log( 1 + np.exp(xbMatrix)))- (alpha / 2) * np.sum(w**2), -((gradValy - gradVal) - gradPenalty)


In [9]:
## implementation of grad_check, gradient ascent and optimizeFn borrowed from HW1
def grad_check(f,xy0,delta=1e-6,tolerance=1e-7):
    f0,g0 = f(xy0)
    p = len(xy0)
    finite_diff = np.zeros(p)
    gradient_correct = True
    for i in range(p):
        xy1 = np.copy(xy0)
        xy2 = np.copy(xy0)
        xy1[i] = xy1[i] - 0.5*delta
        xy2[i] = xy2[i] + 0.5*delta
        f1,_ = f(xy1)
        f2,_ = f(xy2)
        finite_diff = (f2 - f1)/(delta)
        if (abs(finite_diff - g0[i])>tolerance):
            print("Broken partial",i," Finite Diff: ",finite_diff," Partial: ",g0[i])
            gradient_correct = False
    return gradient_correct

g = lambda xy0: loglikelihood(xy0, wordArray, output, 1)
grad_check( g, weights, delta=1e-6, tolerance=1e-5 )

True

In [10]:
def gradient_ascent(f,x,init_step,iterations):  
    f_val,grad = f(x)                           
    f_vals = [f_val]
    for it in range(iterations):                
        done = False                            
        line_search_it = 0                      
        step = init_step                        
        while not done and line_search_it<100: 
            new_x = x + step*grad               
            new_f_val,new_grad = f(new_x)      
            if new_f_val<f_val:                 
                step = step*0.95                
                line_search_it += 1            
            else:
                done = True                     
        
        if not done:                            
            print("Line Search failed.")
        else:
            f_val = new_f_val                   
            x = new_x
            grad = new_grad
            f_vals.append(f_val)
    return f_val, x

def optimizeFn( init_step, iterations, alpha, w):
    g = lambda xy0: loglikelihood(xy0, wordArray, output, alpha)
    f_val, update_w = gradient_ascent( g, w, init_step, iterations )
    return f_val, update_w


In [11]:
print(loglikelihood(weights, wordArray, output, 300))
f_val, update_w=optimizeFn( init_step = 1e-7, iterations=200, alpha=300, w = weights) #set init_step to 1e-4, 1e-5, 1e-6
print(loglikelihood(update_w, wordArray, output, 300))


(3152.4983043630805, array([ 6871.97843022, 12305.81451137,   981.70587337,  2177.12895215,
          18.39575132,    82.88372366]))
(14073.189947869465, array([14304.45670606, 23690.66033857,  1945.96930249,  4117.20216874,
         144.28065841,   288.91080261]))


In [12]:
def prediction(w, Xf):
    wf = np.array(list(map(float, w)))
    prob = np.exp(np.dot(Xf, wf))/(1 + np.exp(np.dot(Xf,wf)))
    res = np.zeros(Xf.shape[0])
    res[prob >= .5] = 0
    res[prob >= .5] = 1
    return res
loss = 0

check = prediction(weights, wordArray) ## checking number of misclassifications with untrained weights
for k in range(len(check)):
    if output[k] != check[k]:
        loss += 1
        
print(loss)
loss = 0

check = prediction(update_w, wordArray) ## checking number of misclassifications with trained weights
for k in range(len(check)):
    if output[k] != check[k]:
        loss += 1
        
print(loss)

3891
4440


In [13]:
## It doesn't look like gradient ascent is doing very well. Let's try the other way.
def gradient_descent(f,x,init_step,iterations):  
    f_val,grad = f(x)                           
    f_vals = [f_val]
    for it in range(iterations):                
        done = False                            
        line_search_it = 0                      
        step = init_step                        
        while not done and line_search_it<100: 
            new_x = x - step*grad               
            new_f_val,new_grad = f(new_x)      
            if new_f_val>f_val:                 
                step = step*0.95                
                line_search_it += 1            
            else:
                done = True                     
        
        if not done:                            
            print("Line Search failed.")
        else:
            f_val = new_f_val                   
            x = new_x
            grad = new_grad
            f_vals.append(f_val)
    return f_val, x

def optimizeFn( init_step, iterations, alpha, w):
    g = lambda xy0: loglikelihood(xy0, wordArray, output, alpha)
    f_val, update_w = gradient_descent( g, w, init_step, iterations )
    return f_val, update_w


In [14]:
f_val, update_w=optimizeFn( init_step = 1e-7, iterations=200, alpha=300, w = weights)

In [15]:
loss = 0
print(weights)
check = prediction(weights, wordArray) ##Checking number of misclassifications with un-trained weights
for k in range(len(check)):
    if output[k] != check[k]:
        loss += 1
        
print(loss)
loss = 0

check = prediction(update_w, wordArray) ## Checking number of misclassifications with trained weights.
for k in range(len(check)):
    if output[k] != check[k]:
        loss += 1
        
print(loss)

[-1.08036291e-03  7.66315042e-04  9.61938498e-04  6.38333722e-04
  4.05485940e-05 -5.57014930e-05]
3891
102


In [16]:
## There we go. It looks like lower weights are better. Lets try starting with higher initial weights.
weights2 = np.ones(wordArray.shape[1])
f_val, update_w2=optimizeFn( init_step = 1e-5, iterations=500, alpha= 0, w = weights2)
loss = 0

check = prediction(update_w2, wordArray)
for k in range(len(check)):
    if output[k] != check[k]:
        loss += 1

print(loss)

111


In [17]:
## Pretty good. Lets try with another data set.
filename4 = "V.STORM VS EG2.txt"

path_to_file4 = os.path.join(base_path, filename4)
fd4 = open(path_to_file4, 'r', encoding ="utf8")

testList = fd4.read().split('\n')


for k in range(len(testList)):
    testList[k] = testList[k].split(' ')
    m = m + 1
    
wordl2 = testList.copy()
words2 = []
    
## removes the timestamp and the user name, which are stored in the first two elements of each sub-array
for k in range(len(wordl2)):
    wordl2[k] = wordl2[k][2:]

## adds each word to the words array. A word will only be added once, regardless of how many times it appears
for k in range(len(wordl2)):
    for w in wordl2[k]:
        if words2.count(w) == 0:
            words2.append(w)

## creates room for parameters, add more zeros as needed
## characters in the word is set in index 2
for k in range(len(words2)):
    words2[k] = [words2[k], 0, len(words2[k]), 0, 0, 0, 0]
    
tList = putTogether(words2, wordl2)

In [18]:
emoteList.append("sheriffDANCE")
emoteList.append("sheriffDANCE2")
emoteList.append("sheriffDANCE3")
emoteList.append("dayPwr")
emoteList.append("dayHot")
emoteList.append("singsingF")
emoteList.append("singsingU")
emoteList.append("singsingC")
emoteList.append("singsingK")
emoteList.append("pokiAww")
emoteList.append("pokiLUL")
emoteList.append("pokiYAY")
emoteList.append("pokiBAN")
emoteList.append("pokiPOP")
emoteList.append("pokiHug")
emoteList.append("rtzW1")
emoteList.append("rtzW2")
emoteList.append("forsenPosture")
emoteList.append("forsenPosture1")
emoteList.append("forsenPosture2")
emoteList.append("forsenTake")
emoteList.append("forsenA")
emoteList.append("forsenDED")

In [19]:
tList2 = tList.copy()
for k in range(len(tList2)): 
    tList2[k] = deque(tList2[k])
    tList2[k].popleft()
    
wordArray2 = np.asarray(tList2)         

## Creates a 4542 x 1 Matrix to hold outputs
output2 = np.ones(wordArray2.shape[0])

for k in range(output2.shape[0]):
    if emoteList.count(words2[k][0]) >= 1:
        output2[k] = 1
    else:
        output2[k] = 0   
        
weights2 = np.random.randn(wordArray.shape[1])*0.001 ## some random weights to test against

In [20]:
loss = 0
check = prediction(weights2, wordArray2)
for k in range(len(check)):
    if output2[k] != check[k]:
        loss += 1

print(loss)

loss = 0

check = prediction(update_w2, wordArray2)
for k in range(len(check)):
    if output2[k] != check[k]:
        loss += 1

print(loss)

243
451


In [21]:
## Seems pretty good.
print(output2.size)

13762


# Logistic regression -- log-likelihood for $0,1$ labels
$$
\renewcommand{\xx}{\mathbf{x}}
\renewcommand{\yy}{\mathbf{y}}
\renewcommand{\zz}{\mathbf{z}}
\renewcommand{\vv}{\mathbf{v}}
\renewcommand{\bbeta}{\boldsymbol{\mathbf{\beta}}}
\renewcommand{\loglik}{\mathcal{LL}}
\renewcommand{\penloglik}{\mathcal{PLL}}
\renewcommand{\likelihood}{\mathcal{L}}
\renewcommand{\Data}{\textrm{Data}}
\renewcommand{\given}{ \big| }
\renewcommand{\MLE}{\textrm{MLE}}
\renewcommand{\tth}{\textrm{th}}
\renewcommand{\Gaussian}[2]{\mathcal{N}\left(#1,#2\right)}
\renewcommand{\norm}[1]{\left\lVert#1\right\rVert}
\renewcommand{\ones}{\mathbf{1}}
\renewcommand{\diag}[1]{\textrm{diag}\left( #1 \right)}
\renewcommand{\sigmoid}[1]{\sigma\left(#1\right)}
\renewcommand{\myexp}[1]{\exp\left\{#1\right\}}
\renewcommand{\mylog}[1]{\log\left\{#1\right\}}
$$


Probability of a single sample is when $y \in \{0,1\}$:
$$
p(y\given\xx,\beta_0,\beta) = \frac{\myexp{y(\beta_0 + \xx^T\beta)}}{1 + \myexp{(\beta_0 + \xx^T\beta)}}
$$

Likelihood function is:
$$
\likelihood(\beta_0,\beta\given\yy,\xx) = \prod_i \frac{\myexp{y_i(\beta_0 + \xx_i^T\beta)}}{1 + \myexp{(\beta_0 + \xx^T\beta)}}
$$

Log-likelihood function is:
$$
\loglik(\beta_0,\beta\given\yy,\xx) = \sum_i y_i(\beta_0 + \xx_i^T\beta) - \log\left\{1 + \myexp{(\beta_0 + \xx_i^T\beta)} \right\}
$$

Ridge regularized log-likelihood:
$$
\penloglik(\beta_0,\beta\given\yy,\xx) =  \sum_i y_i(\beta_0 + \xx_i^T\beta) - \log\left\{1 + \myexp{(\beta_0 + \xx_i^T\beta)} \right\} - \frac{\lambda}{2}\norm{\beta}^2
$$

1. $ L(\mathbf{w}) =  \prod_i \frac{\myexp{y_i(\mathbf{w} \cdot \mathbf{x_i})}}{1 + \myexp{(\mathbf{w} \cdot \mathbf{x_i})}}$
OR $ L(\mathbf{w}) = \prod_i \frac{\myexp{y_i(\beta_0 + \xx_i^T\beta)}}{1 + \myexp{(\beta_0 + \xx^T\beta)}} $


2. $ LL(\mathbf{w}) =  \sum_i y_i(\mathbf{w} \cdot \mathbf{x_i}) - \log\left\{1 + \myexp{(\mathbf{w} \cdot \mathbf{x_i})} \right\} $
OR $ LL(\mathbf{w}) =  \sum_i y_i(\beta_0 + \xx_i^T\beta) - \log\left\{1 + \myexp{(\beta_0 + \xx_i^T\beta)} \right\} $


3. $ PLL(\mathbf{w}) = \sum_i y_i(\mathbf{w} \cdot \mathbf{x_i}) - \log\left\{1 + \myexp{(\mathbf{w} \cdot \mathbf{x_i})} \right\} - \frac{\alpha}{2}\norm{w_j}^2 $
OR $ PLL(\mathbf{w}) = \sum_i y_i(\beta_0 + \xx_i^T\beta) - \log\left\{1 + \myexp{(\beta_0 + \xx_i^T\beta)} \right\} - \frac{\lambda}{2}\norm{\beta}^2 $

In [22]:
## Now feed text message data for clip classifier
## Todo: 1. A list of cleaned data (dimension N*3)
##       2. Create a randomly generated weights list and a list for holding outputs (of the same dimension)
##       3. Run OptimizeFn (use either gradient_ascent OR gradient_descent) to get an updated weights
##       4. Run prediction on both weights
##       5. Write a function to count loss -- compare it with output (in this case, more than average chats & unique users)

In [42]:
#Here is where we create our manually selected highlights for training the clip classifier

output2 = np.zeros(192) ## replace wordArray 2 with the data matrix and output2 with the output matrix for data set 1

manualC = output2
manualC[35] = 1
manualC[36] = 1
manualC[51] = 1
manualC[52] = 1
manualC[84] = 1
manualC[85] = 1
manualC[86] = 1
manualC[94] = 1
manualC[95] = 1
manualC[96] = 1
manualC[115] = 1
manualC[116] = 1
manualC[117] = 1
index = [167, 168, 169, 170, 171, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185]
manualC[index] = 1
print(manualC)
for k in range(len(manualC)):
    if manualC[k] == 1:
        print(k);
print(manualC.shape)

np.savetxt("V.STORM VS EG manual highlights.txt", manualC)
        
#manualC2 = output3
#index = [17, 18, 19, 20, 42, 43, 44, 55, 56, 57, 58, 94, 95, 96, 97, 110, 111, 112, 113, 125, 126, 127, 128, 143, 144, 145, 146, 166, 167, 168, 169, 170, 171, 172]
#manualC2[index] = 1

#np.savetxt("V.STORM VS EG2 manual highlights.txt", manualC)

#manualC3 = output4
#index = [30, 31, 42, 43, 50, 51, 63, 64, 73, 74, 75, 85, 86, 87, 88, 115, 116, 125, 126, 137, 138, 139, 140]
#manualC3[index] = 1

#np.savetxt("V.STORM VS EG3 manual highlights.txt", manualC)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
35
36
51
52
84
85
86
94
95
96
115
116
117
167
168
169
170
171
173
174
175
176
177
178
179
180
181
182
183
184
185
(192,)


In [43]:
## 1. Load data from cleaned clip data 
matrix = np.loadtxt('clipData.txt') #, dtype=int

print(matrix.shape)

(4, 192)


In [44]:
## 2a. Creates a 3 x 1 Matrix to hold weights
weights = np.random.randn(matrix.shape[0])*0.001  #TODO: 1 vs 0 here??
print("weights shape ", weights.shape)
print("weights: ", weights)

## 2b. Creates a N x 1 Matrix for the highlights
#highlights = np.ones(matrix.shape[1])
highlights = np.loadtxt('V.STORM VS EG manual highlights.txt')
print("highlights.shape: ", highlights.shape)

print(highlights)

#TODO: Check shape

data = matrix.copy()
data = data.T

print("data shape: ", data.shape)

bxMatrix = np.dot(weights, matrix)
print("bxMatrix.shape", bxMatrix.shape)
ybxMatrix = highlights * bxMatrix

xbMatrix = np.dot(matrix.T, weights)

weights shape  (4,)
weights:  [-5.29989725e-04 -1.16536353e-03  1.11527741e-03  3.50235225e-05]
highlights.shape:  (192,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
data shape:  (192, 4)
bxMatrix.shape (192,)


In [45]:
## 3a. Run OptimizeFn using gradient_ascent 

def loglikelihoodNew(w, X, y, alpha):
    xbMatrix = np.dot(X, w) #TODO: (w,X) or (X,w) or (X.T, w)
    prob = np.exp(y * xbMatrix)/(1 + np.exp(xbMatrix))
    X = X.T
    gradVal = np.dot(X, np.exp(xbMatrix)/(1 + np.exp(xbMatrix)))
    gradValy = np.dot(X, y)
    penalty = alpha/2 * np.sum(w**2)
    gradPenalty = -alpha * w
    return -np.sum(y * (xbMatrix) - np.log( 1 + np.exp(xbMatrix)))- (alpha / 2) * np.sum(w**2), -((gradValy - gradVal) - gradPenalty)

def optimizeFnAsc( init_step, iterations, alpha, w, data, highlight):
    g = lambda xy0: loglikelihoodNew(xy0, data, highlight, alpha)
    f_val, update_asc_w = gradient_ascent( g, w, init_step, iterations )
    return f_val, update_asc_w

f_val_asc, update_asc_w=optimizeFnAsc( init_step = 1e-7, iterations=400, alpha=100, w = weights, data=data, highlight=highlights)

print(update_asc_w)
print(f_val_asc)

[6.09207488e-02 8.75444443e-02 5.87282038e-02 3.48837075e-05]
527.6651904681283


In [46]:
## 3b. Run OptimizaFn using gradient_descent
def optimizeFnDes( init_step, iterations, alpha, w, data, highlight):
    g = lambda xy0: loglikelihoodNew(xy0, data, highlight, alpha)
    f_val, update_des_w = gradient_descent( g, w, init_step, iterations )
    return f_val, update_des_w

f_val_des, update_des_w=optimizeFnDes( init_step = 1e-7, iterations=200, alpha=300, w = weights, data=data, highlight=highlights)

print(update_des_w)

[-1.01286280e-02 -1.88358225e-02 -7.89851620e-03  3.52342922e-05]


In [47]:
## 4a. Run prediction on untrained weights
print("data.T shape: ", data.shape)

untrained_predictions = prediction(weights, data)

print("prediction shape: ", untrained_predictions.shape)
print(untrained_predictions)

data.T shape:  (192, 4)
prediction shape:  (192,)
[1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0.
 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1.]


In [48]:
## 4b. Run prediction on trained weights
trained_asc_predictions = prediction(update_asc_w, data)

print(trained_asc_predictions.shape)
print(len(trained_asc_predictions))

print(trained_asc_predictions)

## TODO: do it on updated weights using gradient descent
trained_des_predictions = prediction(update_des_w, data)

print(trained_des_predictions.shape)
print(len(trained_des_predictions))

print(trained_des_predictions)

(192,)
192
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
(192,)
192
[1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 

In [50]:
## 5. Compare it against highlights
def getAccuracy(highlights, predictions):
    loss = 0
    for k in range(len(predictions)):
        if highlights[k] != predictions[k]:
            loss += 1
    return loss/len(highlights)

for k in range(len(trained_des_predictions)):
    if highlights[k] != trained_des_predictions[k]:
        loss += 1

lossUn = getAccuracy(highlights, untrained_predictions)
lossAsc = getAccuracy(highlights, trained_asc_predictions)
lossDes = getAccuracy(highlights, trained_des_predictions)
print("accuracy for untrained data: ", lossUn)
print("accuracy for gradient asc trained data: ", lossAsc)
print("accuracy for gradient des trained data: ", lossDes)

accuracy for untrained data:  0.3385416666666667
accuracy for gradient asc trained data:  0.8385416666666666
accuracy for gradient des trained data:  0.3229166666666667


In [51]:
## TODO: run it against the manually classified highlights instead of highlights resulting from the upper 25% of all parameters
# 1. Manually identify the highlight minutes
# 2. Create 0 and 1 matrix for the highlights

In [52]:
# Now let's combine the two models 
# 1. In every minute: run emoClassifier on all the worlds using the 6 parameters and the trained weights
# 2. Multiply the wordcount by y (output from 1.) to get the total amount of emos in each minute
# 3. Use either the result from 2, or the percentage of emos (divide 2 by total amount of words) as the 4th parameter
# 4. With the 4th parameter, run the predictions again

In [53]:
## Function for Running emoModel on the words of each minute 
# (given a double array of words (row: each comment, column: all the words in this comment))
# Here we are returning the total count of emos in this block (a minute)
def runEmoModel(wordsList):
    words = []

    ## adds each word to the words array. A word will only be added once, regardless of how many times it appears
    for k in range(len(wordsList)):
        for w in wordsList[k]:
            if words.count(w) == 0:
                words.append(w)

    ## creates room for parameters, add more zeros as needed
    ## characters in the word is set in index 2
    for k in range(len(words)):
        words[k] = [words[k], 0, len(words[k]), 0, 0, 0, 0]

    weightedWords = putTogether(words, wordsList)
    ## index 1 is set to the number of occurances   
    ## index 2 is set to the length of the word
    ## index 3 is set to the number of letters that are capitalized
    ## index 4 is set to average number of repetitions per message
    ## index 5 is set to the number of lower case letters before the first capitalized letter, zero if there are no capitalizations
    ## or if the entire word is capitalized
    ## index 6 is set to the number of times the word appears in a message by itself.
    
    #print("weightedWords", weightedWords)

    #Remove the first column (word String)
    for k in range(len(weightedWords)): 
        weightedWords[k] = deque(weightedWords[k])
        weightedWords[k].popleft()
    
    wordsMatrix = np.asarray(weightedWords) 
      
    y = prediction(update_w2, wordsMatrix)
    #print("here is the y", y)
    #print(np.count_nonzero(y))
    
    #TODO: now we have the prediction (0 and 1) of each unique word
    #Need to multiply it by the words occurrence 
    count = 0
    for k in range(len(y)):
        if y[k] == 1:
            occur = wordsMatrix[k][0]
            count += occur
    
    return count


wordsListTesting = [['will', 'NA', 'finals', 'be', 'casted', 'here', '?'], ['oh', 'nvm']]

words = [['Yooo']]

runEmoModel(words)

0

In [54]:
import os
import numpy as np

## loads text file into python
base_path = "C:/Users/mckelly/Desktop/562/FinalProject"
#base_path = "/Users/ami_zou/Desktop/COMP562_Project"
filename = "V.STORM VS EG.txt"

path_to_file = os.path.join(base_path, filename)
fd = open(path_to_file, 'r', encoding ="utf8")

## loads the text file into a double array with each message being an index in the first array and each word in the message
## as an index in the second array
lines = fd.read().split('\n')

# timestamps = ['h:mm:ss', 'h:mm:ss']
timestamps = []
# users = ['username for message 0', 'username for message 1',...]
users = []
# messages = ['message 0 without timestamp or user', 'message 1 without timestamp or user'...]
messages = []


for line in lines:
    
    timestamps.append(line[1:8])
    
    #linestr = create string out of the line
    linestr = ''.join(line)
    
    #split strings into users and messages based on '<' and '>' characters
    users.append(''.join(linestr[linestr.find('<')+1:linestr.find('>')-1]))
    messages.append(linestr[linestr.find('>')+2:])
    
#trim empty line that is at the end of each file
timestamps = timestamps[0:len(timestamps)-1]
messages = messages[0:len(messages)-1]
users = users[0:len(users)-1]

#get last timestamp
last_timestamp = timestamps[len(timestamps)-1]

#convert last time stamp into number of rows for data matrix
#we need one row for each minute of the stream, even if there is no activity in that minute

#TODO: change it to 15sec!!!
num_of_rows = ((int(last_timestamp[0])*60 + int(last_timestamp[2:4])) + 1)*4

# number of columns in our data matrix
num_of_columns = 4

#initialize matrix to 0. using floats to get average length of message
matrix = np.zeros((num_of_columns, num_of_rows))

# users_per_minutes = { key = minute : value = list of distinct users that spoke in that minute}
users_per_minute = dict()

# messages_per_minute = { key = minute : value = list of messages in that minute}
messages_per_minute = dict()

# words_per_minute = { key = minue : value = list of the words in that minue}
words_per_minute = dict()

for i in range(0, len(timestamps)):
    
    # convert timestamp into index of matrix
    hour = int(timestamps[i][0])
    minute = int(timestamps[i][2:4])
    sec15 = int(timestamps[i][5:7])
    index = (hour*60 + minute)*4
    
    # Update index every 15 seconds
    if(sec15 <= 30 and sec15 > 15):
        index += 1
    elif(sec15 <= 45):
        index += 2
    elif(sec15 <= 59):
        index += 3
    
    # column [0] is just a counter of how many messages were in that minute
    matrix[0][index] +=1
    
    # building messages_per_minute dictionary...
    if index in messages_per_minute.keys():
        messages_per_minute[index].append(''.join(messages[i]))
    else:
        messages_per_minute[index] = list()
        messages_per_minute[index].append(''.join(messages[i]))  
        
    # building words_per_minute dictionary...
    words = messages[i].split()
    if index in words_per_minute.keys():
        words_per_minute[index].append(words)
    else:
        words_per_minute[index] = list()
        words_per_minute[index].append(words) #messages[i].split()
    
    # building users_per_minute dictionary...
    if index in users_per_minute.keys():
        if (''.join(users[i])) not in users_per_minute[index]:
            users_per_minute[index].append(''.join(users[i]))
    else:
        users_per_minute[index] = list()
        users_per_minute[index].append(''.join(users[i]))
    
       
# convert dictionaries above into matrix
for i in range(0, len(matrix[2])):
    
    # calculate avg length of message before inserting into matrix
    if i in messages_per_minute.keys():
        tmp = 0
        for x in messages_per_minute[i]:
            tmp += len(x)
        
        matrix[1][i] = tmp / len(messages_per_minute[i])
        
    # already trimmed duplicate users, so just insert length of users into matrix
    if i in users_per_minute.keys():
        matrix[2][i] = int(len(users_per_minute[i]))
        
    # feed word lists to the emoModel
#    if i in words_per_minute.keys():
#        words = words_per_minute[i]
#        #print("words for this minute ", words)
#        emoC = runEmoModel(words)
#        matrix[3][i] = emoC

#saves matrix to a txt file
np.savetxt('clipData.txt', matrix) #, dtype=int


for i in range(0, len(matrix[0])):
    print(i,":\t", matrix[0][i], "\t", matrix[1][i], "\t", matrix[2][i], matrix[3][i])
# matrix formatting
# each row corresponds to one minute of the chatlog, starting at minute 0
# column [0] = how many messages were in that minute
# column [1] = is average length in characters of each message in that minute
# column [2] = is number of users in that minute
# column [3] = the number/percentage of emos in the data

0 :	 0.0 	 0.0 	 0.0 0.0
1 :	 17.0 	 15.0 	 17.0 0.0
2 :	 31.0 	 22.774193548387096 	 29.0 0.0
3 :	 10.0 	 59.2 	 10.0 0.0
4 :	 0.0 	 0.0 	 0.0 0.0
5 :	 12.0 	 25.583333333333332 	 12.0 0.0
6 :	 21.0 	 17.857142857142858 	 19.0 0.0
7 :	 13.0 	 25.692307692307693 	 13.0 0.0
8 :	 0.0 	 0.0 	 0.0 0.0
9 :	 10.0 	 18.5 	 10.0 0.0
10 :	 37.0 	 17.81081081081081 	 30.0 0.0
11 :	 23.0 	 19.47826086956522 	 22.0 0.0
12 :	 0.0 	 0.0 	 0.0 0.0
13 :	 15.0 	 23.933333333333334 	 15.0 0.0
14 :	 35.0 	 25.457142857142856 	 29.0 0.0
15 :	 20.0 	 16.0 	 18.0 0.0
16 :	 0.0 	 0.0 	 0.0 0.0
17 :	 10.0 	 16.2 	 9.0 0.0
18 :	 21.0 	 19.571428571428573 	 20.0 0.0
19 :	 10.0 	 28.3 	 10.0 0.0
20 :	 0.0 	 0.0 	 0.0 0.0
21 :	 10.0 	 31.5 	 10.0 0.0
22 :	 14.0 	 25.357142857142858 	 14.0 0.0
23 :	 31.0 	 19.29032258064516 	 30.0 0.0
24 :	 0.0 	 0.0 	 0.0 0.0
25 :	 7.0 	 30.714285714285715 	 7.0 0.0
26 :	 24.0 	 21.166666666666668 	 20.0 0.0
27 :	 4.0 	 15.5 	 4.0 0.0
28 :	 0.0 	 0.0 	 0.0 0.0
29 :	 6.0 	 28.8333

In [55]:
## Now with the 4th parameter, we can run all the predictions again":

## 2a. Creates a 4 x 1 Matrix to hold weights
weightsNew = np.random.randn(matrix.shape[0])*0.001  #TODO: 1 vs 0 here??
print("weights shape ", weightsNew.shape)
print("weights: ", weightsNew)

## 2b. Creates a N x 1 Matrix for the highlights
#highlights = np.ones(matrix.shape[1])
highlightsNew = np.loadtxt('manual_highlights.txt')
print("highlights.shape: ", highlightsNew.shape)

print(highlightsNew)

#TODO: Check shape

dataNew = matrix.copy()
dataNew = dataNew.T

print("dataNew shape: ", dataNew.shape)

#bxMatrix = np.dot(weights, matrix)
#ybxMatrix = highlights * bxMatrix

#xbMatrix = np.dot(matrix.T, weights)

weights shape  (4,)
weights:  [-0.00012025 -0.00164312 -0.00100848 -0.0014599 ]
highlights.shape:  (298,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1.
 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0.
 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1.
 1. 0. 0. 0. 0. 1.

In [56]:
## 3a. Run OptimizeFn using gradient_ascent 

def loglikelihoodNew(w, X, y, alpha):
    xbMatrix = np.dot(X, w) #TODO: (w,X) or (X,w) or (X.T, w)
    prob = np.exp(y * xbMatrix)/(1 + np.exp(xbMatrix))
    X = X.T
    gradVal = np.dot(X, np.exp(xbMatrix)/(1 + np.exp(xbMatrix)))
    gradValy = np.dot(X, y)
    penalty = alpha/2 * np.sum(w**2)
    gradPenalty = -alpha * w
    return -np.sum(y * (xbMatrix) - np.log( 1 + np.exp(xbMatrix)))- (alpha / 2) * np.sum(w**2), -((gradValy - gradVal) - gradPenalty)

def optimizeFnAsc( init_step, iterations, alpha, w, data, highlight):
    g = lambda xy0: loglikelihoodNew(xy0, data, highlight, alpha)
    f_val, update_asc_w = gradient_ascent( g, w, init_step, iterations )
    return f_val, update_asc_w

f_val_asc_new, update_asc_w_new =optimizeFnAsc( init_step = 1e-7, iterations=400, alpha=100, w = weights, data=dataNew, highlight=highlights)

print("update_asc_w_new: ", update_asc_w_new)
print("f_val_asc_new: ", f_val_asc_new)

## 3b. Run OptimizaFn using gradient_descent
def optimizeFnDes( init_step, iterations, alpha, w, data, highlight):
    g = lambda xy0: loglikelihoodNew(xy0, data, highlight, alpha)
    f_val, update_des_w = gradient_descent( g, w, init_step, iterations )
    return f_val, update_des_w

f_val_des_new, update_des_w_new =optimizeFnDes( init_step = 1e-7, iterations=200, alpha=300, w = weights, data=dataNew, highlight=highlights)

print("update_des_w_new: ",update_des_w_new)
print("f_val_des_new: ", f_val_des_new)

## 4a. Run prediction on untrained weights
untrained_predictions_new = prediction(weights, dataNew)

print("prediction_new shape: ", untrained_predictions_new.shape)
print("untrain_new: ", untrained_predictions_new)

## 4b. Run prediction on trained weights
trained_asc_predictions_new = prediction(update_asc_w_new, dataNew)

print(trained_asc_predictions_new.shape)
print(len(trained_asc_predictions_new))

print("asc_new: ", trained_asc_predictions_new)

## TODO: do it on updated weights using gradient descent
trained_des_predictions_new = prediction(update_des_w_new, dataNew)

print(trained_des_predictions_new.shape)
print(len(trained_des_predictions_new))

print("des_new: ", trained_des_predictions_new)

## 5. Compare it against highlights
def getAccuracy(highlights, predictions):
    loss = 0
    for k in range(len(predictions)):
        if highlights[k] != predictions[k]:
            loss += 1
    return loss/len(highlights)

for k in range(len(trained_des_predictions)):
    if highlights[k] != trained_des_predictions[k]:
        lossDes += 1

lossUn_new = getAccuracy(highlights, untrained_predictions_new)
lossAsc_new = getAccuracy(highlights, trained_asc_predictions_new)
lossDes_new = getAccuracy(highlights, trained_des_predictions_new)
print("accuracy for untrained_new data: ", lossUn_new)
print("accuracy for gradient asc_new trained data: ", lossAsc_new)
print("accuracy for gradient des_new trained data: ", lossDes_new)

update_asc_w_new:  [6.09207488e-02 8.75444443e-02 5.87282038e-02 3.48837075e-05]
f_val_asc_new:  527.6651904681283
update_des_w_new:  [-1.01286280e-02 -1.88358225e-02 -7.89851620e-03  3.52342922e-05]
f_val_des_new:  106.50915893901812
prediction_new shape:  (192,)
untrain_new:  [1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0.
 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1.]
(192,)
192
asc_new:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.